In [2]:
import pandas as pd
import numpy as np

import random

import time
start_time = time.perf_counter()

from datetime import date

from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
import math

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler

from sklearn.inspection import permutation_importance

from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.decomposition import PCA

from sklearn import preprocessing

from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import altair as alt

import altair as alt
from vega_datasets import data

world = alt.topo_feature(data.world_110m.url, feature='countries')

import warnings
# SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead#
# warnings.filterwarnings("ignore", message="A value is trying to be set on a copy of a slice from a DataFrame.")
warnings.filterwarnings("ignore", message='''A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead''')

#https://data.imf.org/?sk=388dfa60-1d26-4ade-b505-a05a558d9a42&sId=1479329132316

course_file_path = '/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/IMF/'

imf_target_values_import = pd.read_csv('/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/imf_gdp_pc_target_values.csv')

file_model_df = imf_target_values_import.copy()

file_model_df = file_model_df#.iloc[1:,:]
file_model_df['Country Name'] = file_model_df['GDP per capita, current prices\n (U.S. dollars per capita)']
file_model_df.index = file_model_df['Country Name']
file_model_df=file_model_df.drop(columns = ['Country Name','GDP per capita, current prices\n (U.S. dollars per capita)'])
file_model_df.set_axis([int(float(x)) for x in file_model_df.columns],axis = 'columns', inplace=True)

country_exclusion_list = [
'Africa (Region)',
'Asia and Pacific',
'Australia and New Zealand',
'Caribbean',
'Central America',
'Central Asia and the Caucasus',
'East Asia',
'Eastern Europe ', 
'Europe',
'Middle East (Region)',
'North Africa',
'North America',
'Pacific Islands ', 
'South America',
'South Asia',
'Southeast Asia',
'Sub-Saharan Africa (Region) ',
'Western Europe',
'Western Hemisphere (Region)',
'ASEAN-5',
'Advanced economies',
'Emerging and Developing Asia',
'Emerging and Developing Europe',
'Emerging market and developing economies',
'Euro area',
'European Union',
'Latin America and the Caribbean',
'Major advanced economies (G7)',
'Middle East and Central Asia',
'Other advanced economies',
'Sub-Saharan Africa',
'World',
'nan',
'NaN',
'©IMF, 2022'
]

model_df = file_model_df.copy()
model_df = model_df[~model_df.index.isin(country_exclusion_list)]
model_df=model_df.fillna(0).replace('no data',0)
model_df = model_df.loc[model_df.index.notna()]

completed_file_dfs = [
          'BOPAGG_02-04-2023 07-59-39-92_timeSeries/BOPAGG_02-04-2023 07-59-39-92_timeSeries.csv'#3
         ,'CDIS_01-15-2023 18-58-45-21_timeSeries/CDIS_01-15-2023 18-58-45-21_timeSeries.csv'
         ,'CPI_02-07-2023 16-19-35-85_timeSeries/CPI_02-07-2023 16-19-35-85_timeSeries.csv'#6
         ,'ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv'#8
         ,'EQ_02-04-2023 18-05-06-26_timeSeries/EQ_02-04-2023 18-05-06-26_timeSeries.csv'#9
         ,'FAS_02-04-2023 13-20-29-76_timeSeries/FAS_02-04-2023 13-20-29-76_timeSeries.csv'#10
         ,'FDI_07-14-2022 10-45-25-27_timeSeries/FDI_07-14-2022 10-45-25-27_timeSeries.csv'#11
         ,'FISCALDECENTRALIZATION_11-13-2022 06-45-12-25_timeSeries/FISCALDECENTRALIZATION_11-13-2022 06-45-12-25_timeSeries.csv'
         ,'FM_02-07-2023 04-43-40-20_timeSeries/FM_02-07-2023 04-43-40-20_timeSeries.csv'#13
         ,'FSI_02-03-2023 08-19-04-78_timeSeries/FSI_02-03-2023 08-19-04-78_timeSeries.csv'#14
         ,'FSIRE_02-04-2023 08-23-40-70_timeSeries/FSIRE_02-04-2023 08-23-40-70_timeSeries.csv'#15
         ,'GENDER_EQUALITY_01-25-2023 14-59-11-92_timeSeries/GENDER_EQUALITY_01-25-2023 14-59-11-92_timeSeries.csv'#16
         ,'GFSCOFOG_02-07-2023 04-44-34-74_timeSeries/GFSCOFOG_02-07-2023 04-44-34-74_timeSeries.csv'
         ,'GFSE_02-07-2023 04-44-54-86_timeSeries/GFSE_02-07-2023 04-44-54-86_timeSeries.csv'#18
         ,'GFSFALCS_02-07-2023 04-45-22-02_timeSeries/GFSFALCS_02-07-2023 04-45-22-02_timeSeries.csv'
         ,'GFSIBS_02-06-2023 14-42-18-39_timeSeries/GFSIBS_02-06-2023 14-42-18-39_timeSeries.csv'
         ,'GFSMAB_02-07-2023 04-46-16-64_timeSeries/GFSMAB_02-07-2023 04-46-16-64_timeSeries.csv'
         ,'GFSR_02-07-2023 04-46-55-75_timeSeries/GFSR_02-07-2023 04-46-55-75_timeSeries.csv'
         ,'GFSSSUC_02-07-2023 04-47-28-17_timeSeries/GFSSSUC_02-07-2023 04-47-28-17_timeSeries.csv'
         ,'HPDD_04-11-2020 23-37-06-60_timeSeries/HPDD_04-11-2020 23-37-06-60_timeSeries.csv'#24
         ,'IFS_02-07-2023 12-51-31-57_timeSeries/IFS_02-07-2023 12-51-31-57_timeSeries.csv'#25
         ,'IRFCL_02-07-2023 08-35-05-78_timeSeries/IRFCL_02-07-2023 08-35-05-78_timeSeries.csv'#26
         ,'PCTOT_01-31-2023 17-02-07-08_timeSeries/PCTOT_01-31-2023 17-02-07-08_timeSeries.csv'#29
         ,'WHDREO_02-06-2023 03-53-04-14_timeSeries/WHDREO_02-06-2023 03-53-04-14_timeSeries.csv' #31
         ,'WoRLD_02-08-2023 01-04-45-48_timeSeries/WoRLD_02-08-2023 01-04-45-48_timeSeries.csv' #32
         ] 


train_year = 2020
inputdate = '2023-04-04'

train_y_df =model_df[[train_year+1]].replace('no data',0).fillna(0)
test_y_df =model_df[[train_year+2]].replace('no data',0).fillna(0)

completed_file_dfs_sample = [
              'BOPAGG_02-04-2023 07-59-39-92_timeSeries/BOPAGG_02-04-2023 07-59-39-92_timeSeries.csv'#3
         ,'CDIS_01-15-2023 18-58-45-21_timeSeries/CDIS_01-15-2023 18-58-45-21_timeSeries.csv'
        #  ,'CPI_02-07-2023 16-19-35-85_timeSeries/CPI_02-07-2023 16-19-35-85_timeSeries.csv'#6
        #  ,'ED_02-07-2023 04-41-04-68_timeSeries/ED_02-07-2023 04-41-04-68_timeSeries.csv'#8
        # ,'FISCALDECENTRALIZATION_11-13-2022 06-45-12-25_timeSeries/FISCALDECENTRALIZATION_11-13-2022 06-45-12-25_timeSeries.csv'
        # ,'FM_02-07-2023 04-43-40-20_timeSeries/FM_02-07-2023 04-43-40-20_timeSeries.csv'#13
        #  ,'FSI_02-03-2023 08-19-04-78_timeSeries/FSI_02-03-2023 08-19-04-78_timeSeries.csv'#14
        #  ,'FSIRE_02-04-2023 08-23-40-70_timeSeries/FSIRE_02-04-2023 08-23-40-70_timeSeries.csv'#15
        #  ,'GENDER_EQUALITY_01-25-2023 14-59-11-92_timeSeries/GENDER_EQUALITY_01-25-2023 14-59-11-92_timeSeries.csv'#16
        #  ,'GFSCOFOG_02-07-2023 04-44-34-74_timeSeries/GFSCOFOG_02-07-2023 04-44-34-74_timeSeries.csv'
        #  ,'GFSE_02-07-2023 04-44-54-86_timeSeries/GFSE_02-07-2023 04-44-54-86_timeSeries.csv'#18
        #  ,'GFSFALCS_02-07-2023 04-45-22-02_timeSeries/GFSFALCS_02-07-2023 04-45-22-02_timeSeries.csv'
        #  ,'GFSIBS_02-06-2023 14-42-18-39_timeSeries/GFSIBS_02-06-2023 14-42-18-39_timeSeries.csv'
        #  ,'GFSMAB_02-07-2023 04-46-16-64_timeSeries/GFSMAB_02-07-2023 04-46-16-64_timeSeries.csv'
        #  ,'GFSR_02-07-2023 04-46-55-75_timeSeries/GFSR_02-07-2023 04-46-55-75_timeSeries.csv'
        #  ,'GFSSSUC_02-07-2023 04-47-28-17_timeSeries/GFSSSUC_02-07-2023 04-47-28-17_timeSeries.csv'
        #  ,'HPDD_04-11-2020 23-37-06-60_timeSeries/HPDD_04-11-2020 23-37-06-60_timeSeries.csv'#24
        #  ,'IFS_02-07-2023 12-51-31-57_timeSeries/IFS_02-07-2023 12-51-31-57_timeSeries.csv'#25
        #  ,'IRFCL_02-07-2023 08-35-05-78_timeSeries/IRFCL_02-07-2023 08-35-05-78_timeSeries.csv'#26
        #  ,'PCTOT_01-31-2023 17-02-07-08_timeSeries/PCTOT_01-31-2023 17-02-07-08_timeSeries.csv'#29
        #  ,'WoRLD_02-08-2023 01-04-45-48_timeSeries/WoRLD_02-08-2023 01-04-45-48_timeSeries.csv' #32
            ]

k10_all_lr_results = pd.DataFrame(columns = ['r_score','rmse_score'])
k10_all_dt_results = pd.DataFrame(columns = ['r_score','rmse_score'])
k10_all_mlp_results = pd.DataFrame(columns = ['r_score','rmse_score'])
k10_all_rf_results = pd.DataFrame(columns = ['r_score','rmse_score'])

pca_train_df = pd.DataFrame(index = train_y_df.index)
pca_test_df = pd.DataFrame(index = test_y_df.index)

meta_dict = {}


In [3]:

for file_name in completed_file_dfs:
# for file_name in completed_file_dfs_sample:
    # start_time = time.perf_counter()
    print(file_name)
    print(course_file_path+file_name.split('/')[0]+'/'+file_name.split('_')[0])
    df_file = pd.read_csv(course_file_path+file_name.split('/')[0]+'/'+file_name.split('_')[0]+'_df.csv')
    meta_of_df = pd.read_csv(course_file_path+file_name.split('/')[0]+'/metadata_'+file_name.split('/')[1])
    meta_subject = meta_of_df[meta_of_df['Metadata Attribute']=='Dataset']['Metadata Value']
    print(meta_subject)
    meta_dict[file_name.split('_')[0]] = meta_subject[1]

    meta_dict['ED'] = 'Export Diversification (ED)'
    meta_dict['EQ'] = 'Export Quality (EQ)'
    meta_dict['FSI'] = 'Financial Soundness Indicators (FSI)'
    meta_dict['FSIRE'] = 'Financial Soundness Indicators Reporting Entities (FSI)'
    meta_dict['GFSCOFOG'] = 'Government Financial Statistics Expenditure by Function of Government (GFSCOFOG)'
    meta_dict['GFSE'] = 'Government Financial Statistics Expense (GFSE)'
    meta_dict['GFSFALCS'] = 'Government Financial Statistics Financial Assets and Liabilities Counterpart (GFSFALCS)'
    meta_dict['GFSIBS'] = 'Government Financial Statistics Integrated Balance Sheet (GFSIBS)'
    meta_dict['GFSMAB'] = 'Government Financial Statistics Main Aggregates and Balances (GFSMAB)'
    meta_dict['GFSR'] = 'Government Financial Statistics Revenue (GFSR)'
    meta_dict['GFSSSUC'] = 'Government Financial Statistics Statement of Sources and Uses of Cash (GFSSSUC)'

    file_index = completed_file_dfs.index(file_name)

    df_file = df_file[['Country Name','Country Code','Indicator Name','Indicator Code','Year','Feature Value','filename']]
    column_exclusion_list = ['Xgdppc']
    df_file = df_file[~df_file['Indicator Code'].isin(column_exclusion_list)]

    train_data_df = df_file[df_file['Year'] <= train_year]
    train_data_df = train_data_df[['Country Name','Indicator Code','Feature Value']]
    train_data_df = pd.pivot_table(train_data_df, values = 'Feature Value'
                                   ,index = ['Country Name']
                                   ,columns = ['Indicator Code']
                                   ,aggfunc=np.mean).fillna(0).astype(float)


    test_data_df = df_file[df_file['Year'] <= train_year+1]
    test_data_df = test_data_df[['Country Name','Indicator Code','Feature Value']]
    test_data_df = test_data_df[['Country Name','Indicator Code','Feature Value']]
    test_data_df = pd.pivot_table(test_data_df, values = 'Feature Value'
                                   ,index = ['Country Name']
                                   ,columns = ['Indicator Code']
                                   ,aggfunc=np.mean).fillna(0).astype(float)
    

    train_data_df = train_y_df.merge(train_data_df  , left_on='Country Name', right_on='Country Name', how='inner')#.reset_index()
    train_X = train_data_df.drop(columns =[train_year+1])
    
    
    test_data_df = test_y_df.merge(test_data_df  , left_on='Country Name', right_on='Country Name', how='inner')#.reset_index()
    test_X = test_data_df.drop(columns =[train_year+2])
    test_X = test_data_df
    

    feature_list = [x for x in train_X.columns]
    feature_list = [x for x in feature_list if x in test_X.columns]

    train_X = train_X[feature_list].astype(float)
    test_X = test_X[feature_list].astype(float)

    print(train_X.shape)


    train_y = train_data_df[[train_year+1]].astype(float)
    test_y = test_data_df[[train_year+2]].astype(float)


    train_y_label_encoded = preprocessing.LabelEncoder().fit_transform(train_y)
    test_y_label_encoded = preprocessing.LabelEncoder().fit_transform(test_y)

    ci = .95
    svd_solver = 'auto'#'full'
    nca_pca = PCA(n_components=ci, svd_solver=svd_solver)
    nca_pca.fit(train_X, train_y_label_encoded)

    print(nca_pca.get_params(deep=True))


    print(nca_pca.transform(train_X).shape)
    print(nca_pca.components_.shape)

    folder_train_pca_df = pd.DataFrame(nca_pca.transform(train_X)
                                 ,columns = [file_name.split('_')[0]+'_component_'+str(x) for x in range(nca_pca.components_.shape[0])]
                                 ,index=train_X.index)
    folder_test_pca_df = pd.DataFrame(nca_pca.transform(test_X)
                                 ,columns = [file_name.split('_')[0]+'_component_'+str(x) for x in range(nca_pca.components_.shape[0])]
                                 ,index=test_X.index)

    pca_train_df = pd.concat([pca_train_df, folder_train_pca_df],axis= 1).fillna(0)
    pca_test_df = pd.concat([pca_test_df, folder_test_pca_df],axis= 1).fillna(0)
    
    kf10 = KFold(n_splits = 10)
    kf10.get_n_splits(train_X)


    k10_fold_lr_results = pd.DataFrame(columns = ['r_score','rmse_score'])
    k10_fold_dt_results = pd.DataFrame(columns = ['r_score','rmse_score'])
    k10_fold_mlp_results = pd.DataFrame(columns = ['r_score','rmse_score'])
    k10_fold_rf_results = pd.DataFrame(columns = ['r_score','rmse_score'])

    for i, (train_index, test_index) in enumerate(kf10.split(train_X)):
        print(f"Fold {i}:")

        lr_rgr = LinearRegression().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
        lr_init_regr_score = lr_rgr.score(folder_train_pca_df.iloc[test_index,:],train_y.iloc[test_index,:])
        lr_regr_predict = lr_rgr.predict(folder_test_pca_df)
        lr_rmse = round(math.sqrt(mean_squared_error(test_y, lr_regr_predict)),3)

        k10_fold_lr_results = pd.concat([k10_fold_lr_results,pd.DataFrame([[lr_init_regr_score,lr_rmse]],columns = ['r_score','rmse_score'])],axis=0)

        dt_rgr = DecisionTreeRegressor(max_features= 'auto', max_depth= 25, ccp_alpha = .25).fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
        dt_init_regr_score = dt_rgr.score(folder_train_pca_df.iloc[test_index,:],train_y.iloc[test_index,:])
        dt_regr_predict = dt_rgr.predict(folder_test_pca_df)
        rmse = round(math.sqrt(mean_squared_error(test_y, dt_regr_predict)),3)


        k10_fold_dt_results = pd.concat([k10_fold_dt_results,pd.DataFrame([[dt_init_regr_score,rmse]],columns = ['r_score','rmse_score'])],axis=0)
    
        mlp_rgr = MLPRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
        mlp_rgr_score = mlp_rgr.score(folder_train_pca_df.iloc[test_index,:],train_y.iloc[test_index,:])
        mlp_rgr_predict = mlp_rgr.predict(folder_test_pca_df)
        rmse = round(math.sqrt(mean_squared_error(test_y, mlp_rgr_predict)),3)

        k10_fold_mlp_results = pd.concat([k10_fold_mlp_results,pd.DataFrame([[mlp_rgr_score,rmse]],columns = ['r_score','rmse_score'])],axis=0)
        rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
        rf_rgr_score = rf_rgr.score(folder_train_pca_df.iloc[test_index,:],train_y.iloc[test_index,:])
        rf_rgr_predict = rf_rgr.predict(folder_test_pca_df)
        rmse = round(math.sqrt(mean_squared_error(test_y, rf_rgr_predict)),3)

        k10_fold_rf_results = pd.concat([k10_fold_rf_results,pd.DataFrame([[rf_rgr_score,rmse]],columns = ['r_score','rmse_score'])],axis=0)

    print(lr_rgr.__class__.__name__)

    k10_all_lr_results=pd.concat([k10_all_lr_results, pd.DataFrame([[k10_fold_lr_results.mean(axis=0)[0],k10_fold_lr_results.mean(axis=0)[1]]]
                                                                   ,columns = ['r_score','rmse_score'])],axis=0)
                                  
    print(dt_rgr.__class__.__name__)

    k10_all_dt_results=pd.concat([k10_all_dt_results, pd.DataFrame([[k10_fold_dt_results.mean(axis=0)[0],k10_fold_dt_results.mean(axis=0)[1]]]
                                                                   ,columns = ['r_score','rmse_score'])],axis=0)

    print(mlp_rgr.__class__.__name__)

    k10_all_mlp_results=pd.concat([k10_all_mlp_results, pd.DataFrame([[k10_fold_mlp_results.mean(axis=0)[0],k10_fold_mlp_results.mean(axis=0)[1]]]
                                                                   ,columns = ['r_score','rmse_score'])],axis=0)
                                   
    print(rf_rgr.__class__.__name__)  

    k10_all_rf_results=pd.concat([k10_all_rf_results, pd.DataFrame([[k10_fold_rf_results.mean(axis=0)[0],k10_fold_rf_results.mean(axis=0)[1]]]
                                                                   ,columns = ['r_score','rmse_score'])],axis=0)

    print(k10_fold_lr_results )
    print(k10_fold_dt_results )
    print(k10_fold_mlp_results )
    print(k10_fold_rf_results )

   


BOPAGG_02-04-2023 07-59-39-92_timeSeries/BOPAGG_02-04-2023 07-59-39-92_timeSeries.csv
/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/IMF/BOPAGG_02-04-2023 07-59-39-92_timeSeries/BOPAGG
1    Balance of Payments (BOP)
Name: Metadata Value, dtype: object
0
(187, 37)
{'copy': True, 'iterated_power': 'auto', 'n_components': 0.95, 'random_state': None, 'svd_solver': 'auto', 'tol': 0.0, 'whiten': False}
(187, 1)
(1, 37)
                    BCA_BP6_GDP_PT   BCA_BP6_USD  BFDA_BP6_USD  BFDL_BP6_USD  \
Country Name                                                                   
Afghanistan             -14.935138 -2.888089e+09  9.204503e+06  7.409852e+07   
Albania                  -9.876298 -1.222538e+09  8.009669e+07  9.890653e+08   
Algeria                   1.788512  1.337286e+09  1.416074e+08  1.622333e+09   
Andorra                   0.000000  5.085633e+08  1.174413e+08  3.474519e+08   
Angola                    5.324749  4.01761

/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_33471/2801162816.py:177: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])


RandomForestRegressor
0.17369753674056432
12567.78
Fold 1:
LinearRegression -0.021019445360950018 19100.052
DecisionTreeRegressor
DecisionTreeRegressor -3.7741384894949404 14100.161
MLPRegressor
-27685829019.747387
5381490822.553
RandomForestRegressor
-2.3666199388384705
13119.847
Fold 2:
LinearRegression 0.09972350638320682 19111.576
DecisionTreeRegressor
DecisionTreeRegressor -0.5461158291584516 13963.791
MLPRegressor
-61417440926.71395
17156353213.149


/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_33471/2801162816.py:177: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/7l/v039n9y14dl1p58v4l2n14_r

RandomForestRegressor
-0.16120189504609428
12895.01
Fold 3:
LinearRegression 0.3846989387369478 19101.02
DecisionTreeRegressor
DecisionTreeRegressor -0.425960443586352 13613.148
MLPRegressor
-8171821345665.624
62633486671.309
RandomForestRegressor
-0.026085024675936097
13116.279
Fold 4:
LinearRegression 0.32826807184515483 19113.833
DecisionTreeRegressor
DecisionTreeRegressor 0.018338369897503037 12749.544
MLPRegressor
-1391980329283.956
32749262751.731
RandomForestRegressor
0.31577677904537993


...

/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_33471/2801162816.py:177: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilaye

 -0.03227006115491515 16792.734
DecisionTreeRegressor
DecisionTreeRegressor -0.4776165023501153 9157.81
MLPRegressor
-1.0453592175413942
27420.558
RandomForestRegressor
-0.19809837321612056
10022.345
Fold 8:
LinearRegression 0.5328374925157549 16662.314
DecisionTreeRegressor
DecisionTreeRegressor 0.7160801263312903 4886.73


/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_33471/2801162816.py:177: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilaye

MLPRegressor
-0.30642419542771315
27412.398
RandomForestRegressor
0.6281570344147942
8270.481
Fold 9:
LinearRegression 0.15313622924955972 16653.804
DecisionTreeRegressor
DecisionTreeRegressor -1.0153905585505352 8632.475
MLPRegressor
-0.504503932885672
27414.76
RandomForestRegressor
0.5248538707476315
8315.591
LinearRegression
DecisionTreeRegressor
MLPRegressor
RandomForestRegressor
    r_score rmse_score
0  0.521315  16682.641
0 -0.648654  16817.701
0  0.826498  16655.061
0  0.703891  16662.807
0  0.403161  16674.986
0  0.309455   16701.48
0  0.658789  16670.336
0  -0.03227  16792.734
0  0.532837  16662.314
0  0.153136  16653.804
    r_score rmse_score
0  0.352565   5398.445
0  0.119539   4496.611
0   0.77456    3842.68
0 -0.324675   8158.082
0 -2.255713  13930.224
0  0.140689   8487.564
0   0.22421   7009.142
0 -0.477617    9157.81
0   0.71608    4886.73
0 -1.015391   8632.475
    r_score rmse_score
0 -0.714726  27416.943
0 -0.456143  27419.317
0 -0.693767  27418.247
0 -0.621628  27

/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/7l/v039n9y14dl1p58v4l2n14_r0000gn/T/ipykernel_33471/2801162816.py:177: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_rgr = RandomForestRegressor().fit(folder_train_pca_df.iloc[train_index,:], train_y.iloc[train_index,:])


In [4]:
print(k10_all_lr_results )
print(k10_all_dt_results )
print(k10_all_mlp_results )
print(k10_all_rf_results )

print(k10_all_lr_results.mean(axis=0))    
print(k10_all_dt_results.mean(axis=0))    
print(k10_all_mlp_results.mean(axis=0))    
print(k10_all_rf_results.mean(axis=0)) 

                  r_score             rmse_score
0               -0.579402             19574.8744
0                -7.44058             20606.5027
0               -0.112452             22164.6307
0                0.054947             21309.2675
0                0.020275             21389.9243
0               -0.054068             22247.9682
0                0.645339             12217.1947
0               -6.749911             26115.7151
0                0.058741             20963.6495
0     -76691651177.176666         757899242.7226
0              -29.047445             30448.8068
0                0.117313             20487.8662
0                  -0.174             22910.0207
0               -0.188794             22615.6697
0 -34853236159047294976.0  15636087556236.869141
0               -0.851736             23403.1219
0               -5.500748             25964.4436
0               -0.157884              22589.971
0               -0.241035             22653.5023
0               -0.0